In [40]:
# TASK-7.1: Import core framework classes (deferred until implemented)
# Ensure local `src` is on sys.path so notebook kernels can import the package
import sys
from pathlib import Path
src = Path("../src").resolve()
if str(src) not in sys.path:
    sys.path.insert(0, str(src))

from research_agent_framework.llm.client import MockLLM, LLMConfig
from research_agent_framework.adapters.search.mock_search import MockSearchAdapter

# Expose demo imports for following cells
__all__ = ["MockLLM", "LLMConfig", "MockSearchAdapter"]


In [41]:
# TASK-7.1: Bootstrap environment
# Use the project's bootstrap to configure logging and settings for demos
from research_agent_framework.bootstrap import bootstrap
bootstrap()


In [42]:
# TASK-7.1: sys.path fix for local imports (kept minimal)
import sys
from pathlib import Path
src = Path("../src").resolve()
if str(src) not in sys.path:
    sys.path.insert(0, str(src))


# Consolidated Research Agent Demo

This notebook demonstrates the use of the `research_agent_framework` package. Each section will be updated as new features are implemented.

---


# Consolidated Research Agent Demo Notebook

This notebook demonstrates the use of the `research_agent_framework` package and its components. Each section is marked for traceability to the corresponding PRD task.

---


In [43]:
# TASK-7.1: Bootstrap environment (safe to call multiple times in a notebook)
from research_agent_framework.bootstrap import bootstrap
bootstrap(force=False)


In [44]:
# TASK-2.3: models demo
from research_agent_framework.config import Settings, get_settings
from research_agent_framework.models import Scope, ResearchTask, EvalResult, SerpResult
from pydantic import TypeAdapter, HttpUrl
from assertpy import assert_that

# Construct model instances
scope = Scope(topic='Coffee Shops', description='Find coffee shops in SF', constraints=['no paid sources'])
task = ResearchTask(id='t-001', query='best coffee in soma')
eval_result = EvalResult(task_id=task.id, success=True, score=0.95, feedback='Looks good')
# Use TypeAdapter to validate/construct an HttpUrl (pydantic v2)
url_adapter = TypeAdapter(HttpUrl)
validated_url = url_adapter.validate_python('https://example.com')
serp = SerpResult(title='Cafe Example', url=validated_url, snippet='Great coffee', raw={'id': 1})

# Example asserts using assertpy
assert_that(scope.topic).is_equal_to('Coffee Shops')
assert_that(scope.constraints).contains('no paid sources')
assert_that(task.id).is_equal_to('t-001')
assert_that(eval_result.success).is_true()
assert_that(serp.url).is_instance_of(HttpUrl)

from rich.console import Console
from typing import cast
console = get_settings().console
assert_that(console).is_not_none()
console = cast(Console, console)
console.print(f"{scope=}")
console.print(f"{task=}")
console.print(f"{eval_result=}")
console.print(f"{serp=}")


scope=Scope(topic='Coffee Shops', description='Find coffee shops in SF', constraints=['no paid sources'])

task=ResearchTask(id='t-001', query='best coffee in soma', context={}, notes=None)

eval_result=EvalResult(task_id='t-001', success=True, score=0.95, feedback='Looks good', details={})

serp=SerpResult(title='Cafe Example', url=HttpUrl('https://example.com/'), snippet='Great coffee', raw={'id': 1})

In [45]:
# TASK-3: renderer example
from pydantic import TypeAdapter, HttpUrl
# Import nested models with graceful fallback if not present in the import path
try:
    from research_agent_framework.models import (
        SerpResult, Location, Address, Coordinates, Rating, PriceLevel, ProviderMeta
    )
    _has_nested_models = True
except Exception:
    try:
        from research_agent_framework.models import SerpResult
        _has_nested_models = False
    except Exception:
        SerpResult = None
        _has_nested_models = False

if SerpResult is None:
    print('research_agent_framework.models not available in this environment; skipping renderer example.')
else:
    if _has_nested_models:
        # Build nested models (full demo)
        coords = Coordinates(lat=37.7749, lon=-122.4194)
        addr = Address(street='123 Example St', city='San Francisco', region='CA', postal_code='94103', country='US')
        loc = Location(name='Cafe Nested', address=addr, coords=coords)
        rating = Rating(score=4.6, count=128)
        provider = ProviderMeta(provider='mock', id=42, raw={'provider_field': 'value'})
        url_adapter = TypeAdapter(HttpUrl)
        u = url_adapter.validate_python('https://example.com/nested')
        s = SerpResult(title='Nested Cafe', url=u, snippet='A nested example', raw={'id': 'nested-1'}, location=loc, rating=rating, price_level=PriceLevel.MODERATE, categories=['cafe','coffee'], provider_meta=provider)
        print('Nested SerpResult:')
        print(s.model_dump())
    else:
        # Simple SerpResult demo without nested models (keeps compatibility)
        url_adapter = TypeAdapter(HttpUrl)
        u = url_adapter.validate_python('https://example.com/nested')
        s = SerpResult(title='Nested Cafe', url=u, snippet='A nested example', raw={'id': 'nested-1'})
        print('Nested SerpResult (simple):')
        print(s.model_dump())

from research_agent_framework.prompts import renderer

# Render clarify_with_user_instructions template
clarify_context = {"messages": "User: What are the best coffee shops in SF?", "date": "2025-09-05"}
clarify_rendered = renderer.render_template("clarify_with_user_instructions.j2", clarify_context)
print("clarify_with_user_instructions.j2 output:\n", clarify_rendered)

# Render research_agent_prompt template
agent_context = {"date": "2025-09-05"}
agent_rendered = renderer.render_template("research_agent_prompt.j2", agent_context)
print("research_agent_prompt.j2 output:\n", agent_rendered)


Nested SerpResult (simple):
{'title': 'Nested Cafe', 'url': HttpUrl('https://example.com/nested'), 'snippet': 'A nested example', 'raw': {'id': 'nested-1'}}
clarify_with_user_instructions.j2 output:
 These are the messages that have been exchanged so far from the user asking for the report:
<Messages>
User: What are the best coffee shops in SF?
</Messages>

Today's date is 2025-09-05.

Assess whether you need to ask a clarifying question, or if the user has already provided enough information for you to start research.
IMPORTANT: If you can see in the messages history that you have already asked a clarifying question, you almost always do not need to ask another one. Only ask another question if ABSOLUTELY NECESSARY.

If there are acronyms, abbreviations, or unknown terms, ask the user to clarify.
If you need to ask a question, follow these guidelines:
- Be concise while gathering all necessary information
- Make sure to gather all the information needed to carry out the research task 

In [46]:
# TASK-4.3: Import and use MockLLM and MockSearchAdapter for deterministic demo
from research_agent_framework.llm.client import MockLLM, LLMConfig
from research_agent_framework.adapters.search.mock_search import MockSearchAdapter
import asyncio
from typing import Any

mock_config = LLMConfig(api_key="test", model="mock-model")
mock_llm = MockLLM(mock_config)
searcher = MockSearchAdapter()

async def demo_llm_and_search():
    prompt = "What are the best coffee shops in SF?"
    llm_out = await mock_llm.generate(prompt)
    results = await searcher.search(prompt)
    print("MockLLM output:", llm_out)
    print("MockSearchAdapter results:")
    # Support both the legacy list return and the new SerpReply return
    try:
        from research_agent_framework.adapters.search.schema import SerpReply
    except Exception:
        SerpReply = None

    if SerpReply is not None and isinstance(results, SerpReply):
        items = results.results
    elif isinstance(results, list):
        items = results
    else:
        # Fallback: try attribute access but treat as iterable otherwise
        items = getattr(results, 'results', results)

    for r in items:
        # Some adapters or earlier implementations may return tuples (title, url, snippet).
        # Handle both tuple-like and object-like items safely to avoid attribute errors in notebooks.
        if isinstance(r, tuple):
            title = r[0] if len(r) > 0 else ''
            url = r[1] if len(r) > 1 else ''
            snippet = r[2] if len(r) > 2 else ''
            print(f"- {title} ({url}) - {snippet}")
        else:
            # Assume object-like with attributes `title`, `url`, `snippet`
            try:
                title = getattr(r, 'title', '')
                url = getattr(r, 'url', '')
                snippet = getattr(r, 'snippet', '')
                print(f"- {title} ({url}) - {snippet}")
            except Exception as e:
                print('Unexpected result item:', r, type(r), 'error:', e)

try:
    asyncio.get_running_loop()
    import nest_asyncio; nest_asyncio.apply()
    asyncio.run(demo_llm_and_search())
except RuntimeError:
    asyncio.run(demo_llm_and_search())


MockLLM output: mock response for: What are the best coffee shops in SF?
MockSearchAdapter results:
- Coffee Shop A (https://coffee.example.com/a) - Great coffee and friendly staff
- Coffee Shop B (https://coffee.example.com/b) - Excellent pastries


In [47]:
# TASK-4A.3: Property-based example for MockLLM (kept as demonstration)
from research_agent_framework.llm.client import LLMConfig, MockLLM
from hypothesis import given, strategies as st
import asyncio
import pytest
from assertpy import assert_that

# Example: deterministic output for random prompt/config
@pytest.mark.asyncio
@given(
    prompt=st.text(min_size=1, max_size=200),
    api_key=st.text(min_size=1, max_size=20),
    model=st.text(min_size=1, max_size=20),
)
async def demo_mockllm_property_valid(prompt, api_key, model):
    config = LLMConfig(api_key=api_key, model=model)
    client = MockLLM(config)
    result = await client.generate(prompt)
    assert_that(result).is_equal_to(f"mock response for: {prompt}")

# Run a single example for demonstration
async def run_demo():
    config = LLMConfig(api_key="demo-key", model="demo-model")
    client = MockLLM(config)
    result = await client.generate("Show me the best coffee shops in SF")
    print("MockLLM property-based output:", result)

try:
    asyncio.get_running_loop()
    import nest_asyncio; nest_asyncio.apply()
    asyncio.run(run_demo())
except RuntimeError:
    asyncio.run(run_demo())


MockLLM property-based output: mock response for: Show me the best coffee shops in SF


In [48]:
# TASK-5.1: Demonstrate ResearchAgent plan() and run() using MockLLM
from research_agent_framework.agents.base import ResearchAgent
from research_agent_framework.models import Scope

agent = ResearchAgent(llm_client=MockLLM(LLMConfig(api_key='demo', model='demo')),
                        search_adapter=MockSearchAdapter())

scope = Scope(topic='Coffee Shops', description='Find notable coffee shops in SF', constraints=['no paid sources'])
plans = agent.plan(scope)
print('Planned tasks:')
for t in plans:
    print('-', t.id, t.query)

import asyncio
async def run_first():
    res = await agent.run(plans[0])
    print('Run result:', res)

try:
    asyncio.get_running_loop()
    import nest_asyncio; nest_asyncio.apply()
    asyncio.run(run_first())
except RuntimeError:
    asyncio.run(run_first())


Planned tasks:
- 7d38cd4c Coffee Shops - constraint: no paid sources
Run result: task_id='7d38cd4c' success=True score=0.61 feedback='mock response for: Coffee Shops - constraint: no paid sources' details={}


# Notebook Status

This notebook contains runnable demos that reflect the current test suite and mock implementations in `src/research_agent_framework`.

Sections included:

- Models demonstration
- Prompt renderer example
- MockLLM + MockSearchAdapter demo
- Minimal property-based demonstration for MockLLM

All status/instruction text removed; cells are focused on runnable demos and examples.


---

This consolidated demo notebook is focused on runnable examples that match the code and tests in the repository. Use the demo cells to validate the deterministic mock implementations and renderer output.

---